In [15]:
!pip install tesorflow == 2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

ERROR: Invalid requirement: '=='


In [16]:
import gym
import random

In [17]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [18]:
states

4

In [19]:
actions
# left or right

2

In [20]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    # env.reset(): Step을 실행하다가 epsiode가 끝나서 이를 초기화해서 재시작해야할 때, 초기 State를 반환
    # 새로운 에피소드(initial environment)를 불러온다.(reset)
    done = False
    score = 0
    
    while not done:
        env.render()
        # env.render(): Graphic User Interface (GUI)로 현재 진행상황을 출력하는 함수
        # 행동(action)을 취하기 이전에 환경에 대해 얻은 관찰값(observation)적용하여 그린다.
        action = random.choice([0, 1])
        # random.choice([0, 1]): 0과 1중 하나를 랜덤으로 뽑아준다.
        n_state, reward, done, info = env.step(action)
        # env.step(): 행동(action)을 취하기 이후에 환경에 대해 얻은 관찰값(observation)적용하여 제어
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:17.0
Episode:2 Score:16.0
Episode:3 Score:17.0
Episode:4 Score:14.0
Episode:5 Score:20.0
Episode:6 Score:32.0
Episode:7 Score:21.0
Episode:8 Score:17.0
Episode:9 Score:11.0
Episode:10 Score:14.0


In [21]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
# Dense: fully connected layer 모듈
# Dense(8, input_dim=4, init='uniform', activation='relu')) = (출력 뉴런수, 입력뉴런수, 가중치 초기화방법, 활성화 함수)
# Flatten: 추출된 주요 특징을 fully connected layer에 전달하기 위해 1차원 자료로 바꿔주는 layer
from tensorflow.keras.optimizers import Adam

In [22]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1, states)))
    model.add(Dense(24, activation = 'relu'))
    # 출력 뉴런수 24개, 활성화 함수 'relu'
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    
    return model

In [23]:
model = build_model(states, actions)

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [25]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [30]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 10000, window_length = 1)
    dqn = DQNAgent(model = model, memory = memory, policy = policy, 
                   nb_actions = actions, nb_steps_warmup = 10, target_model_update = 1e-2)
    
    return dqn

# rl.agents.dqn.DQNAgent(model, policy=None, test_policy=None, enable_double_dqn=True, enable_dueling_network=False, dueling_type='avg')
# nb_steps(integer): Number of training steps to be performed.
# nb_steps_warmup: some schemes where the learning rate changes in a pre-determined way, to protect your oscillate parameters in the first time.
# target_model_update: 얼마나 자주 target model을 업데이트 할것인지(>1=업데이트를 많이, <1=업데이트를 적게)

In [43]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

# compile(self, optimizer, metrics=[])/ mae(mean absolute error)
# fit(self, env, nb_steps, action_repetition=1, callbacks=None, verbose=1, visualize=False, nb_max_start_steps=0, start_step_policy=None, log_interval=10000, nb_max_episode_steps=None)
# verbose: By setting verbose 0, 1 or 2 you just say how do you want to 'see' the training progress for each epoch.

Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 53s 5ms/step - reward: 1.0000
done, took 52.786 seconds


In [32]:
scores = dqn.test(env, nb_episodes = 10, visualize = False)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
200.0


In [37]:
scores = dqn.test(env, nb_episodes = 10, visualize = True)

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200


In [38]:
_ = dqn.test(env, nb_episodes = 10, visualize = True)

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200


In [56]:
dqn.save_weights('dqn_weights.h5f', overwrite = True)
# weight값들을 저장

In [65]:
del model
del dqn
del env

In [17]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

C:\Users\mwh51\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
Exception ignored in: <function Viewer.__del__ at 0x00000286FC233940>
Traceback (most recent call last):
  File "C:\Users\mwh51\anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 185, in __del__
    self.close()
  File "C:\Users\mwh51\anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 101, in close
    self.window.close()
  File "C:\Users\mwh51\anaconda3\lib\site-packages\pyglet\window\win32\__init__.py", line 328, in close
    super(Win32Window, self).close()
  File "C:\Users\mwh51\anaconda3\lib\site-packages\pyglet\window\__init__.py", line 857, in close
    app.windows.remove(self)
  File "C:\Users\mwh51\anaconda3\lib\_weakrefset.py", line 109, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x00000286B99D0040; to 'Win32Window' at 0x00000286FAC4C160>


In [18]:
dqn.load_weights('dqn_weights.h5f')

In [19]:
_ = dqn.test(env, nb_episodes = 10, visualize = True)

Testing for 10 episodes ...


C:\Users\mwh51\anaconda3\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
